<a href="https://colab.research.google.com/github/TeeganKrieger/ChickenScratch/blob/main/src/Portenta%20H7/Grid%20Recognition%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Recognition Model Training

### Import all needed modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install micromlgen

  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32172 sha256=84ac8838834c4b9b7ad99fe75e58f8823a0b4b2917c60da2daebddc8efc79f08
  Stored in directory: /root/.cache/pip/wheels/6f/98/43/0860b692737af546b9262a8b60ec3690c8bdc51c25e545ae09
Successfully built micromlgen


In [3]:
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import json
from types import SimpleNamespace
import seaborn; seaborn.set()
from keras.callbacks import EarlyStopping
import tensorflow as tf

MODELS_DIR = 'models/'
MODEL_TF = MODELS_DIR + 'model'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'
# TensorFlow is an open source machine learning library

#from micromlgen import port

## Building The Model

In [4]:
model = Sequential()
model.add(Conv2D(8, (3, 2), activation='relu', input_shape=(240, 320, 3)))
model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(16, (3, 2), activation='relu'))
model.add(Conv2D(8, (3, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(9, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 319, 8)       152       
                                                                 
 conv2d_1 (Conv2D)           (None, 236, 318, 16)      784       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 118, 159, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 158, 16)      1552      
                                                                 
 conv2d_3 (Conv2D)           (None, 114, 157, 8)       776       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 57, 78, 8)        0         
 2D)                                                    

## Image and Config Loading

In [5]:
#Loads an image and its cfg file
def importImage(path, X, Y):
    #Load the image
    img = load_img(path)
    arr = img_to_array(img)
    
    #Load the cfg
    configFile = open(path[:-4] + '.cfg')
    configRaw = configFile.read()
    config = json.loads(configRaw, object_hook=lambda d: SimpleNamespace(**d)) 
    
    cfgArr = [config.valid, config.x1, config.y1, config.x2, config.y2, config.x3, config.y3, config.x4, config.y4]
    X.append(arr)
    Y.append(cfgArr)
    return X, Y

In [6]:
def loadAllTrainingData(data_path):
    X = []
    Y = []
    for root, dirs, files in os.walk(data_path, topdown=False):
        for f in files:
            if (f.endswith(".jpg")):
                X, Y = importImage(os.path.join(root, f), X, Y)
    return X, Y

## Training Data Splitting

In [7]:
def splitTrainingValidation(X, Y, split_size=0.2):
    if len(X) == len(Y):
        splitCount = int(len(X) * split_size)
        
        used_indices = []
        
        X_set = []
        X_val = []
        Y_set = []
        Y_val = []
        
        while len(X_val) < splitCount:
            index = random.randint(0, len(X)-1)
            if index not in used_indices:
                X_val.append(X[index])
                Y_val.append(Y[index])
                used_indices.append(index)
        
        for i in range(0, len(X)-1):
            if i not in used_indices:
                X_set.append(X[i])
                Y_set.append(Y[i])
                
        return np.array(X_set), np.array(Y_set), np.array(X_val), np.array(Y_val)

## Loading Training Data and Training

In [8]:
X, Y = loadAllTrainingData("/content/drive/MyDrive/Traing Data Preprocessing/Training Data/Grid")
X, Y, X_val, Y_val = splitTrainingValidation(X, Y, split_size=0.2)

#print(X)
#print(Y)
#print(X_val.shape)
#print(Y_val.shape)

In [9]:
#es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.01)

model.fit(X, Y, epochs=50, validation_data=(X_val, Y_val), batch_size=8)

Epoch 1/50
11/11 [==============================] - 7s 562ms/step - loss: 16390.8457 - accuracy: 0.5349 - val_loss: 4895.5493 - val_accuracy: 0.3333
Epoch 2/50
11/11 [==============================] - 6s 537ms/step - loss: 3794.6367 - accuracy: 0.5000 - val_loss: 2649.6582 - val_accuracy: 0.5238
Epoch 3/50
11/11 [==============================] - 6s 529ms/step - loss: 3189.0894 - accuracy: 0.5116 - val_loss: 2225.7739 - val_accuracy: 0.3810
Epoch 4/50
11/11 [==============================] - 6s 536ms/step - loss: 2662.1304 - accuracy: 0.4651 - val_loss: 2322.4629 - val_accuracy: 0.4286
Epoch 5/50
11/11 [==============================] - 6s 537ms/step - loss: 2685.5403 - accuracy: 0.5698 - val_loss: 1768.8519 - val_accuracy: 0.5238
Epoch 6/50
11/11 [==============================] - 7s 609ms/step - loss: 2036.3933 - accuracy: 0.5930 - val_loss: 1509.2021 - val_accuracy: 0.4762
Epoch 7/50
11/11 [==============================] - 6s 584ms/step - loss: 1562.5034 - accuracy: 0.5349 - val_lo

### Testing Random Images

Try changing the sample number below

In [10]:
X_test = []
Y_test = []
X_test, Y_test = importImage("/content/drive/MyDrive/Traing Data Preprocessing/Training Data/Grid/sample_98.jpg", X_test, Y_test)
X_test = np.array(X_test)
Y_predict = model.predict(X_test)
Y_predict = [[round(Y_predict[0][0]), round(Y_predict[0][1]), round(Y_predict[0][2]), round(Y_predict[0][3]), round(Y_predict[0][4]), round(Y_predict[0][5]), round(Y_predict[0][6]), round(Y_predict[0][7]), round(Y_predict[0][8])]]
print(Y_predict)
print(Y_test)

[[2, 38, 48, 288, 36, 37, 202, 298, 193]]
[[1, 41, 43, 284, 37, 45, 193, 291, 189]]


In [11]:
model.save(MODEL_TF)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops = True


def respresentative_dataset_gen():
    img = tf.data.Dataset.from_tensor_slices(X).batch(1)
    for i in img.take(30):
        yield [i]
        
converter.representative_dataset = respresentative_dataset_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8


model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

INFO:tensorflow:Assets written to: models/model/assets


582272

In [12]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [84.8 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.laun